In [ ]:
notebook/
└── tenhou_source_data/
    ├── tenhou_zip/
    │   ├── scraw2009.zip
    │   ├── scraw2010.zip
    │   └── ...
    ├── tenhou_url_2009_2023/
    │   ├── scc20040101.html
    │   ├── scc20040102.html
    │   ├── scc20040103.html
    │   └── ...
    └── tenhou_haifu/
        ├── tenhou_haifu_2009_05_13_05_15.xml
        ├── tenhou_haifu_2009_05_13_05_30.xml
        ├── tenhou_haifu_2009_05_13_06_45.xml
        └── ...


### `https://tenhou.net/sc/raw/`から手動でダウンロードした年度別zipファイルを解凍し、年度別ファイルを作成

tenhou_zipフォルダ内のzipファイルから鳳凰卓のデータ（scc.html.gz）を解凍・抽出するスクリプト

In [3]:
import zipfile
import gzip
import os
from pathlib import Path

# tenhou_zip フォルダ内の zip ファイルを取得
zip_dir = Path("tenhou_source_data/tenhou_zip")
zip_files = sorted(zip_dir.glob("scraw*.zip"))  # scraw2009.zip〜scraw2023.zip などを対象

# 年の範囲を zip ファイル名から抽出
years = sorted({int(z.name[5:9]) for z in zip_files})
year_range_str = f"{years[0]}_{years[-1]}" if years else "unknown"

# 出力ディレクトリ（年の範囲を含める）
output_root = Path(f"tenhou_source_data/tenhou_url_{year_range_str}")

for zip_path in zip_files:
    zip_file_name = zip_path.name
    year = zip_file_name[5:9]  # "2019" などを抽出
    output_dir = output_root / year
    output_dir.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zf:
        for file_info in zf.infolist():
            if file_info.filename.startswith(f"{year}/scc") and (file_info.filename.endswith(".html.gz") or file_info.filename.endswith(".html")):
                with zf.open(file_info) as file:
                    base_name = os.path.basename(file_info.filename).replace(".gz", "")
                    output_path = output_dir / base_name
                    if file_info.filename.endswith(".html.gz"):
                        with gzip.open(file, 'rb') as html_file:
                            with open(output_path, 'wb') as out_f:
                                out_f.write(html_file.read())
                    else:
                        # .htmlファイルの場合はそのままコピー
                        with open(output_path, 'wb') as out_f:
                            out_f.write(file.read())

print(f"解凍と分類が完了しました（出力先: {output_root}）。")


解凍と分類が完了しました（出力先: tenhou_source_data\tenhou_url_2009_2023）。


### 作成したファイルからHTML内のリンクを抽出し、テーブルデータに整理(フォルダ作成処理が完了していたらここから進める)

tenhou_url_2009_2023をテーブルデータに格納するスクリプト

In [4]:
import os
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup
import re

# === 1. パラメータ指定 ===
file_path = "tenhou_source_data/tenhou_url_2009_2023"
year_range = range(2009, 2024)  # 2009〜2023年
date_range = ("0101", "1231")   # MMDD形式

# 範囲の最初と最後を取得
start_year = year_range[0]
end_year = year_range[-1]

# === 2. ディレクトリ初期化 ===
base_dir = Path(file_path)
selected_files = []

# === 3. 対象ファイルの抽出 ===
for year in year_range:
    year_dir = base_dir / str(year)
    if not year_dir.exists():
        continue
    for file in year_dir.glob("scc*.html"):
        date_str = file.stem[3:11]  # 'scc20200201' → '20200201'
        if date_str.startswith(str(year)):
            mmdd = date_str[4:]
            if date_range[0] <= mmdd <= date_range[1]:
                selected_files.append(file)

# === 4. HTMLからデータ抽出する関数（改良版） ===
def parse_html_file(filepath):
    with open(filepath, encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    # <br>, <br/>, <br /> すべてのパターンに対応して分割
    lines = re.split(r"<br\s*/?>", soup.decode_contents())
    parsed = []

    for line in lines:
        if "href=" not in line:
            continue

        parts = line.split("|")
        if len(parts) < 4:
            continue

        time = parts[0].strip()
        table_type = parts[2].strip()
        url_match = re.search(r'href="([^"]+)"', line)
        url = url_match.group(1) if url_match else None

        # プレイヤーとスコア抽出
        score_part = parts[4] if len(parts) > 4 else ""
        player_scores = re.findall(r'([^\(]+)\(([-+]?\d+(?:\.\d+)?)\)', score_part)

        # 結果を辞書に格納
        record = {
            "file": filepath.name,
            "time": time,
            "table_type": table_type,
            "url": url
        }

        for i in range(4):
            if i < len(player_scores):
                name, score = player_scores[i]
                record[f"player{i+1}"] = name.strip()
                record[f"score{i+1}"] = float(score)
            else:
                record[f"player{i+1}"] = None
                record[f"score{i+1}"] = None

        parsed.append(record)

    return parsed


# === 5. 全ファイルから情報を取得してDataFrame化 ===
all_records = []
for file in selected_files:
    all_records.extend(parse_html_file(file))

dataset_url_df = pd.DataFrame(all_records)

# === 6. 表示（初期テーブル表示） ===
# file列から年月日を抽出して新しい列を追加
dataset_url_df['year'] = dataset_url_df['file'].str.extract(r'(\d{4})').astype(int)
dataset_url_df['month'] = dataset_url_df['file'].str.extract(r'\d{4}(\d{2})').astype(int)
dataset_url_df['day'] = dataset_url_df['file'].str.extract(r'\d{6}(\d{2})').astype(int)

# URLの=を削除
dataset_url_df['url'] = dataset_url_df['url'].str.replace('?log=', 'log/?')

# table_typeの－を削除
dataset_url_df['table_type'] = dataset_url_df['table_type'].str.replace('－', '')

# 日付列を結合して作成
dataset_url_df['date'] = pd.to_datetime(dataset_url_df[['year', 'month', 'day']])


# 表示
pd.set_option('display.max_columns', None)
display(dataset_url_df.head())
# 行と列の数を表示
print(dataset_url_df.shape)


,file,time,table_type,url,player1,score1,player2,score2,player3,score3,player4,score4,year,month,day,date
0,scc20090201.html,03:18,四鳳南喰赤,http://tenhou.net/0/log/?2009020103gm-00a9-000...,EXAMPLE2,47.0,EXAMPLE,1.0,EXAMPLE4,-19.0,EXAMPLE3,-29.0,2009,2,1,2009-02-01
1,scc20090201.html,03:41,四鳳南喰赤,http://tenhou.net/0/log/?2009020103gm-00a9-000...,EXAMPLE,38.0,EXAMPLE2,4.0,EXAMPLE4,-16.0,EXAMPLE3,-26.0,2009,2,1,2009-02-01
2,scc20090220.html,11:10,四鳳南喰赤,http://tenhou.net/0/log/?2009022011gm-00a9-000...,LITHIUM,46.0,三河屋,15.0,三田村茜,-10.0,チャオリ,-51.0,2009,2,20,2009-02-20
3,scc20090220.html,11:15,四鳳東喰赤,http://tenhou.net/0/log/?2009022011gm-00e1-000...,六分儀ゲンドウ,32.0,超ヒモリロ,1.0,NONSTYLE,-13.0,メタファー,-20.0,2009,2,20,2009-02-20
4,scc20090220.html,11:22,四鳳東喰赤,http://tenhou.net/0/log/?2009022011gm-00e1-000...,闘士☆渋川老,37.0,bakase,6.0,平均順位重視,-14.0,AmA,-29.0,2009,2,20,2009-02-20


(3706017, 16)


重複列の確認

In [5]:
# プレイヤー3人の順番を無視して比較するため、新しい列を作る
df = dataset_url_df.copy()

# プレイヤー3人の順番を無視して集合にする
df['player_set'] = df[['player1', 'player2', 'player3']].apply(lambda x: frozenset(x), axis=1)

# 重複判定キー（player_set, date, time の組み合わせ）
df['dup_key'] = list(zip(df['player_set'], df['date'], df['time']))

# 重複行を抽出
duplicated_rows = df[df.duplicated('dup_key', keep=False)]

# 結果表示
display(duplicated_rows)


,file,time,table_type,url,player1,score1,player2,score2,player3,score3,player4,score4,year,month,day,date,player_set,dup_key


### 指定したテーブルデータのリンクから特定のURLをスクレイピングしてXMLファイルを取得する(年度別フォルダ内)

使用するデータセットの指定

In [6]:
# 卓の指定(三人麻雀)
table = '三鳳南喰赤'

# 日付範囲の指定(すべての期間)
start_date = pd.Timestamp('2009-01-01')
end_date = pd.Timestamp('2023-12-31')

# 日付列を結合して作成
dataset_url_df['date'] = pd.to_datetime(dataset_url_df[['year', 'month', 'day']])

# 条件定義
condition_table = dataset_url_df['table_type'] == table
condition_date = dataset_url_df['date'].between(start_date, end_date)
# 不正プレイヤーを含む対戦を除外
condition_player = ~(
    (dataset_url_df['player1'] == 'くうた') | 
    (dataset_url_df['player2'] == 'くうた') | 
    (dataset_url_df['player3'] == 'くうた')
)

# 条件で抽出
dataset_sanma_url_df = dataset_url_df[
    condition_table
    & condition_date  
    & condition_player
]

# 結果を表示
display(dataset_sanma_url_df)

,file,time,table_type,url,player1,score1,player2,score2,player3,score3,player4,score4,year,month,day,date
72,scc20090220.html,17:10,三鳳南喰赤,http://tenhou.net/0/log/?2009022017gm-00b9-000...,俊ころ,44.0,僕、おおくぽん,-10.0,楠下幾太郎,-34.0,None,NaN,2009,2,20,2009-02-20
77,scc20090220.html,17:32,三鳳南喰赤,http://tenhou.net/0/log/?2009022017gm-00b9-000...,楠下幾太郎,35.0,僕、おおくぽん,-3.0,オカルト,-32.0,None,NaN,2009,2,20,2009-02-20
79,scc20090220.html,17:49,三鳳南喰赤,http://tenhou.net/0/log/?2009022017gm-00b9-000...,我々クラス,45.0,ジュン,-10.0,楠下幾太郎,-35.0,None,NaN,2009,2,20,2009-02-20
83,scc20090220.html,18:10,三鳳南喰赤,http://tenhou.net/0/log/?2009022018gm-00b9-000...,楠下幾太郎,46.0,僕、おおくぽん,5.0,我々クラス,-51.0,None,NaN,2009,2,20,2009-02-20
86,scc20090220.html,18:21,三鳳南喰赤,http://tenhou.net/0/log/?2009022018gm-00b9-000...,蜜柑の皮,37.0,楠下幾太郎,-1.0,我々クラス,-36.0,None,NaN,2009,2,20,2009-02-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3705999,scc20231231.html,23:32,三鳳南喰赤,http://tenhou.net/0/log/?2023123123gm-00b9-000...,REBI,42.5,おかあさん,2.9,理論力学,-45.4,None,NaN,2023,12,31,2023-12-31
3706002,scc20231231.html,23:36,三鳳南喰赤,http://tenhou.net/0/log/?2023123123gm-00b9-000...,ホンミン。,54.5,高部軍団,-16.2,パクチうま夫,-38.3,None,NaN,2023,12,31,2023-12-31
3706006,scc20231231.html,23:42,三鳳南喰赤,http://tenhou.net/0/log/?2023123123gm-00b9-000...,樽なの壊さないで,60.0,おかあさん,-15.6,マッスルカズ,-44.4,None,NaN,2023,12,31,2023-12-31
3706009,scc20231231.html,23:46,三鳳南喰赤,http://tenhou.net/0/log/?2023123123gm-00b9-000...,理論力学,51.6,ガーネット,-2.1,らくた,-49.5,None,NaN,2023,12,31,2023-12-31


データフレームのurlから牌譜データ(XML)を取得しtenhou_xml内の年度別フォルダに格納するスクリプト
・minidomで整形
・非同期処理で高速化（asyncio + aiohttp）
・バッチサイズを50に制限(クラッシュ防止)
・アクセスが拒否された場合待機

In [7]:
import pandas as pd
import requests
import os
import xml.dom.minidom

# 年度別フォルダを保存するルートディレクトリ
base_folder = "tenhou_source_data/tenhou_haifu"

# バッチサイズの指定
batch_size = 50

# 各行に対して処理
for i in range(0, len(dataset_sanma_url_df), batch_size):
    batch_df = dataset_sanma_url_df.iloc[i:i + batch_size]

    for idx, row in batch_df.iterrows():
        url = row['url']
        date = row['date']
        times = row['time']
        year = row['year']  # 年度列を利用

        # フォルダ名・ファイル名の作成
        date_str = date.strftime("%Y_%m_%d")
        time_str = times.replace(":", "_")
        year_folder = os.path.join(base_folder, str(year))
        os.makedirs(year_folder, exist_ok=True)
        filename = f"{year_folder}/tenhou_haifu_{date_str}_{time_str}.xml"

        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
        }

        retry_count = 0
        max_retries = 5
        success = False

        while not success and retry_count < max_retries:
            try:
                response = requests.get(url, headers=headers)
                response.encoding = response.apparent_encoding
                response.encoding = "utf-8"
                xml_data = response.text

                dom = xml.dom.minidom.parseString(xml_data)
                pretty_xml = dom.toprettyxml(indent="  ")

                with open(filename, "w", encoding="utf-8") as f:
                    f.write(pretty_xml)

                #print(f"✅ 保存完了！ファイル名: {filename}")
                success = True

            except Exception as e:
                retry_count += 1
                wait_time = 2 ** retry_count
                print(f"❌ エラー（index={idx}）: {e} - リトライします。待機時間: {wait_time}秒")
                time.sleep(wait_time)

    print(f"🕒 バッチ {i // batch_size + 1} 完了。")


🕒 バッチ 1 完了。


KeyboardInterrupt: 